# **Brief description of the problem and data**

This Kaggle competition is about classifying texts taken from Twitter using Natural Language Processing (NLP) to classify which are about real disasters and which are not. 

The complexity of the task is demonstrated using the sample Tweet provided by the competition: "LOOK AT THE NIGHT SKY LAST NIGHT IT WAS ABLAZE" where the author explicitly uses the word “ABLAZE” but means it metaphorically. This is clear to a human right away but it is less clear to a machine that it is not referencing a real disaster.

The dataset consists of 10,000 tweets that were hand classified.

# **Load Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import os
import random
import wordcloud
from wordcloud import WordCloud, STOPWORDS

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.utils import plot_model

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#from functools import reduce
#from nltk.stem import PorterStemmer, WordNetLemmatizer

# **Exploratory Data Analysis (EDA) — Inspect, Visualize and Clean the Data**

In [ ]:
train_data = pd.read_csv('../input/nlp-getting-started/train.csv')
train_data.head()

In [ ]:
test_data = pd.read_csv('../input/nlp-getting-started/test.csv')
test_data.head()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

From a high level look at the datasets it is observed:

* The training set contains 7,613 entries while the testing dataset contains 3,263 entries
* Both datasets contain 4 columns: ID, Keyword, Location, Text
* The training dataset contains 1 additional column which we will be predicting for the test dataset: Target

We will evaluate the fields further below.


In [ ]:
train_data['target'].value_counts().to_frame()

In [ ]:
train_data.groupby(['target'])['target'].count().plot(kind='bar', color = 'blue', title='Target Distribution')
plt.xlabel('Target')
plt.ylabel('Count')

In [ ]:
train_data['keyword'].value_counts()[:10].plot(kind='barh', color='blue')
plt.title("Keywords - Top 10")
plt.xlabel("Count")

In [ ]:
train_data['location'].value_counts()[:10].plot(kind='barh', color='blue')
plt.title("Location - Top 10")
plt.xlabel("Count")

* From the above analysis of the training dataset, we can see approximately 60% of the Tweets reference a non-disaster (target=0) versus a disaster (target=1).
* The keywords field contains a word that represents the overall text.
* The location field contains a geographical location from which the text originated, in some cases a country name while in others a city location.  
* We can consider cleaning up the keyword and location for empty value or standardization yet they will not directly be used in the NLP model below so will defer doing so within the scope of this project.
* Below, we will examine the primary field for our analysis which will be the 'text' derived from the Tweets.

In [ ]:
text_length = train_data[train_data['target']==1]['text'].str.split().map(lambda x: len(x))
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
fig.suptitle('WORDS IN A TWEET')
ax1.hist(text_length, color='red', edgecolor='black')
ax1.set_title('Tweets Disaster')
ax1.set_xlabel('# Words')
ax1.set_ylabel('Frequency')
text_length = train_data[train_data['target']==0]['text'].str.split().map(lambda x: len(x))
ax2.hist(text_length,color='blue', edgecolor='black')
ax2.set_title('Tweets Non-Disaster')
ax2.set_xlabel('# Words')

Above we have a histogram of the average number of words in each text for both disaster and non-disaster related Tweets.
In both cases, the average number of words tends to be approximately 15 with more variance from the average for disaster versus non-disaster tweets.

In [ ]:
positive_cases = "".join(train_data[train_data['target'] == 0]['text'].values)
negative_cases = "".join(train_data[train_data['target'] == 1]['text'].values)

fig, axs = plt.subplots(2, 1, figsize=(20, 8))

wc1 = WordCloud(background_color='white').generate(positive_cases)
wc2 = WordCloud(background_color='white').generate(negative_cases)

axs[0].imshow(wc1, interpolation='bilinear')
axs[0].set_title('Positive')
axs[1].imshow(wc2, interpolation='bilinear')
axs[1].set_title('Negative')

The above word cloud shows the most frequent words for each of our target categories.  While some trends start appearing, the data overall seems 'messy' with an opportunity to clean it up for greater insight and accuracy.  Below we will clean the data by applying lower case and removing:

* Punctation
* Common stopwords  
* Words less than 4 letters
* Non-alphabet characters

In [ ]:
#remove punctuation
def remove_punctuation(x):
    return x.translate(str.maketrans('', '', string.punctuation))

#remove stopwords
def remove_stopwords(x):
    return ' '.join([i for i in x.split() if i not in wordcloud.STOPWORDS])

#remove words less than 4 letters
def remove_less_than(x):
    return ' '.join([i for i in x.split() if len(i) > 3])

#remove words with non-alphabet characters
def remove_non_alphabet(x):
    return ' '.join([i for i in x.split() if i.isalpha()])


In [ ]:
train_data['clean_text'] = train_data['text'].apply(lambda x: x.lower())
train_data['clean_text'] = train_data['clean_text'].apply(remove_less_than)
train_data['clean_text'] = train_data['clean_text'].apply(remove_non_alphabet)
train_data['clean_text'] = train_data['clean_text'].apply(remove_stopwords)
train_data['clean_text'] = train_data['clean_text'].apply(remove_punctuation)
print('done')

In [ ]:
train_data.head(5)

In [ ]:
positive_cases = "".join(train_data[train_data['target'] == 0]['clean_text'].values)
negative_cases = "".join(train_data[train_data['target'] == 1]['clean_text'].values)

fig, axs = plt.subplots(2, 1, figsize=(20, 8))

wc1 = WordCloud(background_color='white').generate(positive_cases)
wc2 = WordCloud(background_color='white').generate(negative_cases)

axs[0].imshow(wc1, interpolation='bilinear')
axs[0].set_title('Positive')
axs[1].imshow(wc2, interpolation='bilinear')
axs[1].set_title('Negative')

With the result of the text cleanup, the above word clouds provide greater insight to the words driving the target characterisation.   The clarity of the cleansed text will be helpful in the below model.  While there are additional opportunities to address mis-spellings and abbreviations within the text further, the current cleansed text will suffice within the scope of this week's assignment.

# **Model Architecture**

We will build a long short term memory network (LSTM) model which is a variety of recurrent neural network discussed during the class lecture.  The LSTM architecture provides a short-term memory for RNN that can last thousands of timesteps, thus "long short-term memory". The intuition behind the LSTM architecture is to create an additional module in a neural network that effectively learns which information might be needed later on in a sequence and when that information is no longer needed - making it a great architecture for natural language processing where we need the network to learn grammatical dependencies.

First, we will tokenize each text sentence. Tokenization will break down the sentences into individual words in order to create a matrix of the relationship between those words.  In totality, this is referred to as a corpus.

In [ ]:
max_features=3000
tokenizer=Tokenizer(num_words=max_features,split=' ')
tokenizer.fit_on_texts(train_data['clean_text'].values)
x = tokenizer.texts_to_sequences(train_data['clean_text'].values)
x = pad_sequences(x)
x.shape

In [ ]:
#XLA (Accelerated Linear Algebra) is a domain-specific compiler for linear algebra that can accelerate TensorFlow models with potentially no source code changes.
# reference: https://www.tensorflow.org/xla
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [ ]:
embed_dim = 32
lstm_out = 32
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = x.shape[1]))
model.add(Dropout(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.4))
model.add(Dense(1,activation='sigmoid'))
adam = optimizers.Adam(learning_rate=0.002)
model.compile(loss = 'binary_crossentropy', optimizer=adam ,metrics = ['accuracy'])
print(model.summary())

# **Results and Analysis**

To evaluate the results, we will split the training data 80% for training the model and 20% to validate the results as shown below:

In [ ]:
y = train_data['target']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 777)

In [ ]:
model.fit(x_train, y_train, epochs = 10, batch_size=32, validation_data=(x_test, y_test))

In [ ]:
y_pred = model.predict(x_test).round()
print(y_pred)

In [ ]:
confuse = confusion_matrix(y_test,y_pred)
plt.figure(figsize=(4, 4))
sns.heatmap(confuse, annot=True, fmt='g', cmap='Blues')

In [ ]:
print(classification_report(y_test, y_pred))

The confusion matrix visualizes and summarizes the performance of the classification:

* Precision, the ratio between the true positives and all the positives, checks the accuracy of the postive class. Out of all the tweets the the model predicted  would be disaster related 75% actual are.
* Recall, the ratio between the number of true positives and number of false negatives, is a measure of the model correctly identifying true positives. Out of all the tweets that were related to disasters, the model predicted this outcome for 81%.
* The f1-score of 78% tells us the model did a pretty good job of predicting if the text was related to true disaster.
* Overall accuracy is 74%

We can adjust the model parameters to attempt to improve the results.   Let's try to increase the learning rate and reduce the drop rate:

In [ ]:
embed_dim = 32
lstm_out = 32
model2 = Sequential()
model2.add(Embedding(max_features, embed_dim,input_length = x.shape[1]))
model2.add(Dropout(0.1))
model2.add(LSTM(lstm_out, dropout=0.1, recurrent_dropout=0.2))
model2.add(Dense(1,activation='sigmoid'))
adam = optimizers.Adam(learning_rate=0.10)
model2.compile(loss = 'binary_crossentropy', optimizer=adam ,metrics = ['accuracy'])
print(model.summary())

In [ ]:
model2.fit(x_train, y_train, epochs = 10, batch_size=32, validation_data=(x_test, y_test))

# **Submission**

In [ ]:
test_data.head()

In [ ]:
test_data['clean_text'] = test_data['text'].apply(lambda x: x.lower())
test_data['clean_text'] = test_data['clean_text'].apply(remove_less_than)
test_data['clean_text'] = test_data['clean_text'].apply(remove_non_alphabet)
test_data['clean_text'] = test_data['clean_text'].apply(remove_stopwords)
test_data['clean_text'] = test_data['clean_text'].apply(remove_punctuation)

l =50
max_features=5000
tokenizer=Tokenizer(num_words=max_features,split=' ')
tokenizer.fit_on_texts(train_data['clean_text'].values)
x = tokenizer.texts_to_sequences(train_data['clean_text'].values)
x = pad_sequences(x, maxlen =l)

tokenizer.fit_on_texts(train_data['clean_text'].values)
test_token = tokenizer.texts_to_sequences(test_data['clean_text'].values)
test_token = pad_sequences(test_token, maxlen =l)

print('done')

In [ ]:
embed_dim = 100
lstm_out = 100
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = x.shape[1]))
model.add(Dropout(0.2))
model.add(LSTM(lstm_out, dropout=0.2, return_sequences=True,recurrent_dropout=0.4))
model.add(Dropout(0.2))
model.add(LSTM(lstm_out,dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
adam = optimizers.Adam(learning_rate=2e-3)
model.compile(loss = 'binary_crossentropy', optimizer=adam ,metrics = ['accuracy'])
print(model.summary())

In [ ]:
#early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(x,y, epochs = 10,validation_split = 0.2 ,batch_size=32) #callbacks=[es_callback], 

In [ ]:
y_hat = model.predict(test_token).round()
submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
submission['target'] = np.round(y_hat).astype('int')
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.groupby(['target'])['target'].count().plot(kind='bar', color = 'blue', title='Target - Test Predictions Distribution')
plt.xlabel('Target')
plt.ylabel('Count')

# **Conclusion**

adfadfadfadf